## Examples with ```CopasiProcess``` using ``builder.Builder`` and ``biosimulator_processes.data_model``

In [1]:
import sys 

sys.path.insert(0, '..')

In [2]:
from builder import Builder, Process
from process_bigraph import pp, pf 
from biosimulator_processes import CORE
from biosimulator_processes.data_model import *
from biosimulator_processes.biosimulator_builder import BiosimulatorBuilder

CobraProcess registered successfully.
CopasiProcess registered successfully.
SmoldynProcess registered successfully.
TelluriumProcess registered successfully.


#### Initialize a fresh instance of the `BiosimulatorBuilder`, which is inherently configured by `biosimulator_processes.CORE`.

In [3]:
b = BiosimulatorBuilder()

In [4]:
###### list registered processes

b.list_processes()

['CobraProcess',
 'CopasiProcess',
 'console-emitter',
 'SmoldynProcess',
 'TelluriumProcess',
 'ram-emitter']

Let's load a BIOMODEL from `basico` and inspect the parameters we can adjust:

#### Example 1: Add a simple `CopasiProcess`-based composite to the bigraph, specifying parameter values through the `data_model`. For now, let's try loading a biomodel and adding a change to initial concentration of a species. We will then add another copasi process instance, whose model is this time loaded from a file (to demonstrate the data model)

Let's define a model instance from a file.

In [5]:
from basico import * 


model_filepath = '../biosimulator_processes/model_files/BIOMD0000000061_url.xml'

os.path.exists(model_filepath)

True

In [6]:
model_from_file = load_model(model_filepath)
get_species(model=model_from_file)

,compartment,type,unit,initial_concentration,initial_particle_number,initial_expression,expression,concentration,particle_number,rate,particle_number_rate,key,sbml_id
name,,,,,,,,,,,,,
Extracellular glucose,extracellular,reactions,mmol/l,6.700000,4.034834e+21,,,6.700000,4.034834e+21,-1.139132e+00,-6.860013e+20,Metabolite_0,GlcX
Cytosolic glucose,cytosol,reactions,mmol/l,0.573074,3.451132e+20,,,0.573074,3.451132e+20,6.820866e+01,4.107622e+22,Metabolite_1,Glc
ATP,cytosol,reactions,mmol/l,2.100000,1.264650e+21,,,2.100000,1.264650e+21,-5.374740e-04,-3.236744e+17,Metabolite_2,ATP
Glucose-6-Phosphate,cytosol,reactions,mmol/l,4.200000,2.529299e+21,,,4.200000,2.529299e+21,2.292073e-05,1.380319e+16,Metabolite_3,G6P
ADP,cytosol,reactions,mmol/l,1.500000,9.033211e+20,,,1.500000,9.033211e+20,9.874740e-04,5.946708e+17,Metabolite_4,ADP
Fructose-6-Phosphate,cytosol,reactions,mmol/l,0.490000,2.950849e+20,,,0.490000,2.950849e+20,-2.357096e-05,-1.419477e+16,Metabolite_5,F6P
"Fructose 1,6-bisphosphate",cytosol,reactions,mmol/l,4.640000,2.794273e+21,,,4.640000,2.794273e+21,3.000184e-05,1.806753e+16,Metabolite_6,FBP
Glyceraldehyde 3-phosphate,cytosol,reactions,mmol/l,0.115000,6.925462e+19,,,0.115000,6.925462e+19,-1.056219e-04,-6.360699e+16,Metabolite_7,GAP
Dihydroxyacetone phosphate,cytosol,reactions,mmol/l,2.950000,1.776532e+21,,,2.950000,1.776532e+21,8.613930e-05,5.187430e+16,Metabolite_8,DHAP


In [7]:
# 1. specify model changes
process_model_changes = ModelChanges(
    species_changes=[SpeciesChanges(name='ADP', initial_concentration=2.1**3)]
)


pp(process_model_changes.model_dump())

{ 'global_parameter_changes': [],
  'reaction_changes': [],
  'species_changes': [ { 'expression': '',
                         'initial_concentration': 9.261000000000001,
                         'initial_expression': '',
                         'initial_particle_number': None,
                         'name': 'ADP',
                         'unit': ''}]}


In [8]:
# 2. define the model schema to be used by the composite process (one of the copasiprocess config parameters)

process_model_from_file = Model(
    model_id='copasi_process_from_file',
    model_source=model_filepath,
    model_name='Simple Composite Process from File',
    model_changes=process_model_changes
)


pp(process_model_from_file.model_dump())

{ 'model_changes': { 'global_parameter_changes': [],
                     'reaction_changes': [],
                     'species_changes': [ { 'expression': '',
                                            'initial_concentration': 9.261000000000001,
                                            'initial_expression': '',
                                            'initial_particle_number': None,
                                            'name': 'ADP',
                                            'unit': ''}]},
  'model_id': 'copasi_process_from_file',
  'model_language': 'sbml',
  'model_name': 'Simple Composite Process from File',
  'model_source': { 'value': '../biosimulator_processes/model_files/BIOMD0000000061_url.xml'},
  'model_units': None}


In [9]:
# 3. Define config schema to be used as 'config' parameter of Process constructor
process_config_from_file = CopasiProcessConfigSchema(
    method='stochastic',
    model=process_model_from_file,
    process_name='simple_copasi_from_file'
)


pp(process_config_from_file.model_dump())

{ 'method': 'stochastic',
  'model': { 'model_changes': { 'global_parameter_changes': [],
                                'reaction_changes': [],
                                'species_changes': [ { 'expression': '',
                                                       'initial_concentration': 9.261000000000001,
                                                       'initial_expression': '',
                                                       'initial_particle_number': None,
                                                       'name': 'ADP',
                                                       'unit': ''}]},
             'model_id': 'copasi_process_from_file',
             'model_language': 'sbml',
             'model_name': 'Simple Composite Process from File',
             'model_source': { 'value': '../biosimulator_processes/model_files/BIOMD0000000061_url.xml'},
             'model_units': None},
  'process_name': 'simple_copasi_from_file'}


In [10]:
process_config_from_file.model

{'model_id': 'copasi_process_from_file',
 'model_source': {'value': '../biosimulator_processes/model_files/BIOMD0000000061_url.xml'},
 'model_language': 'sbml',
 'model_name': 'Simple Composite Process from File',
 'model_changes': {'species_changes': [{'name': 'ADP',
    'unit': '',
    'initial_concentration': 9.261000000000001,
    'initial_particle_number': None,
    'initial_expression': '',
    'expression': ''}],
  'global_parameter_changes': [],
  'reaction_changes': []},
 'model_units': None}

In [11]:
# 4. Add the process instance by the name of 'simple_copasi' to the builder


b[process_config_from_file.process_name].add_process(
    name='CopasiProcess',
    model=process_config_from_file.model,
    method=process_config_from_file.method
)

Let's load a BIOMODEL from `basico` and inspect the parameters we can adjust, and then add it to the composite process:

In [12]:
from basico import *
from basico import biomodels


biomodel_id = 'BIOMD0000000861'
biomodel = load_biomodel(biomodel_id)
get_species(model=biomodel)

,compartment,type,unit,initial_concentration,initial_particle_number,initial_expression,expression,concentration,particle_number,rate,particle_number_rate,key,sbml_id
name,,,,,,,,,,,,,
EpoRJAK2,cyt,reactions,nmol/l,3.976220e+00,9.578143e+14,,,3.976220e+00,9.578143e+14,-0.314737,-7.581552e+13,Metabolite_50,EpoRJAK2
EpoRpJAK2,cyt,reactions,nmol/l,0.000000e+00,0.000000e+00,,,0.000000e+00,0.000000e+00,0.314737,7.581552e+13,Metabolite_51,EpoRpJAK2
p1EpoRpJAK2,cyt,reactions,nmol/l,0.000000e+00,0.000000e+00,,,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,Metabolite_52,p1EpoRpJAK2
p2EpoRpJAK2,cyt,reactions,nmol/l,0.000000e+00,0.000000e+00,,,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,Metabolite_53,p2EpoRpJAK2
p12EpoRpJAK2,cyt,reactions,nmol/l,0.000000e+00,0.000000e+00,,,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,Metabolite_54,p12EpoRpJAK2
EpoRJAK2_CIS,cyt,reactions,nmol/l,0.000000e+00,0.000000e+00,,,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,Metabolite_55,EpoRJAK2_CIS
SHP1,cyt,reactions,nmol/l,2.672510e+01,6.437693e+15,,,2.672510e+01,6.437693e+15,0.000000,0.000000e+00,Metabolite_56,SHP1
SHP1Act,cyt,reactions,nmol/l,0.000000e+00,0.000000e+00,,,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,Metabolite_57,SHP1Act
STAT5,cyt,reactions,nmol/l,7.975350e+01,1.921147e+16,,,7.975350e+01,1.921147e+16,0.000000,0.000000e+00,Metabolite_58,STAT5


In [13]:
reaction_names = get_reactions(model=biomodel)

reaction_names

,scheme,flux,particle_flux,function,key,sbml_id,mapping
name,,,,,,,
reaction_1,EpoRJAK2 -> EpoRpJAK2; Epo SOCS3,0.125895,7.581552e+13,Function for reaction_1,Reaction_48,reaction_1,"{'Epo': 'Epo', 'EpoRJAK2': 'EpoRJAK2', 'JAK2Ac..."
reaction_2,EpoRpJAK2 -> EpoRJAK2; SHP1Act,0.000000,0.000000e+00,Function for reaction_2,Reaction_49,reaction_2,"{'EpoRpJAK2': 'EpoRpJAK2', 'JAK2EpoRDeaSHP1': ..."
reaction_3,EpoRpJAK2 -> p1EpoRpJAK2; SOCS3,0.000000,0.000000e+00,Function for reaction_3,Reaction_50,reaction_3,"{'EpoRActJAK2': 'EpoRActJAK2', 'EpoRpJAK2': 'E..."
reaction_4,EpoRpJAK2 -> p2EpoRpJAK2; EpoRJAK2_CIS SOCS3,0.000000,0.000000e+00,Function for reaction_4,Reaction_51,reaction_4,"{'EpoRActJAK2': 'EpoRActJAK2', 'EpoRCISInh': '..."
reaction_5,p1EpoRpJAK2 -> p12EpoRpJAK2; EpoRJAK2_CIS SOCS3,0.000000,0.000000e+00,Function for reaction_5,Reaction_52,reaction_5,"{'EpoRActJAK2': 'EpoRActJAK2', 'EpoRCISInh': '..."
reaction_6,p2EpoRpJAK2 -> p12EpoRpJAK2; SOCS3,0.000000,0.000000e+00,Function for reaction_6,Reaction_53,reaction_6,"{'EpoRActJAK2': 'EpoRActJAK2', 'SOCS3': 'SOCS3..."
reaction_7,p1EpoRpJAK2 -> EpoRJAK2; SHP1Act,0.000000,0.000000e+00,Function for reaction_7,Reaction_54,reaction_7,"{'JAK2EpoRDeaSHP1': 'JAK2EpoRDeaSHP1', 'SHP1Ac..."
reaction_8,p2EpoRpJAK2 -> EpoRJAK2; SHP1Act,0.000000,0.000000e+00,Function for reaction_8,Reaction_55,reaction_8,"{'JAK2EpoRDeaSHP1': 'JAK2EpoRDeaSHP1', 'SHP1Ac..."
reaction_9,p12EpoRpJAK2 -> EpoRJAK2; SHP1Act,0.000000,0.000000e+00,Function for reaction_9,Reaction_56,reaction_9,"{'JAK2EpoRDeaSHP1': 'JAK2EpoRDeaSHP1', 'SHP1Ac..."


In [14]:
rparam = ReactionParameter(
    parameter_name='EpoRpRJAK2',
    value=21312.2
)

rparam2 = ReactionParameter(
    parameter_name='EpoRJAK2',
    value=21312.2
)

rc1 = ReactionChanges(
    reaction_name='reaction_1',
    parameter_changes=[rparam, rparam2]
)

rc2 = ReactionChanges(
    reaction_name='reaction_11',
    reaction_scheme='A + B -> C'
)

In [15]:
biomodel_process_changes = ModelChanges(
    reaction_changes=[rc1, rc2]
)

In [16]:
pp(biomodel_process_changes.model_dump())

{ 'global_parameter_changes': [],
  'reaction_changes': [ { 'parameter_changes': [ { 'parameter_name': 'EpoRpRJAK2',
                                                   'value': 21312.2},
                                                 { 'parameter_name': 'EpoRJAK2',
                                                   'value': 21312.2}],
                          'reaction_name': 'reaction_1',
                          'reaction_scheme': None},
                        { 'parameter_changes': [],
                          'reaction_name': 'reaction_11',
                          'reaction_scheme': 'A + B -> C'}],
  'species_changes': []}


In [17]:
biomodel_process_model = Model(
    model_id=biomodel_id,
    model_source=biomodel_id,
    model_name='composite_from_biomodel',
    model_changes=biomodel_process_changes
)

pp(biomodel_process_model.model_dump())

{ 'model_changes': { 'global_parameter_changes': [],
                     'reaction_changes': [ { 'parameter_changes': [ { 'parameter_name': 'EpoRpRJAK2',
                                                                      'value': 21312.2},
                                                                    { 'parameter_name': 'EpoRJAK2',
                                                                      'value': 21312.2}],
                                             'reaction_name': 'reaction_1',
                                             'reaction_scheme': None},
                                           { 'parameter_changes': [],
                                             'reaction_name': 'reaction_11',
                                             'reaction_scheme': 'A + B -> C'}],
                     'species_changes': []},
  'model_id': 'BIOMD0000000861',
  'model_language': 'sbml',
  'model_name': 'composite_from_biomodel',
  'model_source': {'value': 'BIOMD000000086

In [18]:
biomodel_process_config = CopasiProcessConfigSchema(
    process_name='copasi_process_from_biomodel',
    method='directMethod',
    model=biomodel_process_model
)


pp(biomodel_process_config.model_dump())

{ 'method': 'directMethod',
  'model': { 'model_changes': { 'global_parameter_changes': [],
                                'reaction_changes': [ { 'parameter_changes': [ { 'parameter_name': 'EpoRpRJAK2',
                                                                                 'value': 21312.2},
                                                                               { 'parameter_name': 'EpoRJAK2',
                                                                                 'value': 21312.2}],
                                                        'reaction_name': 'reaction_1',
                                                        'reaction_scheme': None},
                                                      { 'parameter_changes': [],
                                                        'reaction_name': 'reaction_11',
                                                        'reaction_scheme': 'A '
                                                                 

Add the biomodel config to the bigraph

In [ ]:
b[biomodel_process_config.process_name].add_process(
    name='CopasiProcess',
    model=biomodel_process_config.model,
    method=biomodel_process_config.method
)